In [1]:
!pip install sklearn-crfsuite

import pandas as pd
import nltk
import sklearn
import sklearn_crfsuite
import scipy.stats
import math, string, re
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from itertools import chain
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [3]:
nltk.download('treebank')
wsj = list(nltk.corpus.treebank.tagged_sents())

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\ArrunPersonal\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [4]:
wsj[0]

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

In [5]:
import string
from collections import defaultdict
from utils import get_word_tag
from utils import assign_unk

# def build_vocab(corpus_path):
#     with open(corpus_path, 'r') as f:
#         lines = f.readlines()

#     tokens = [line.split('\t')[0] for line in lines]
#     freqs = defaultdict(int)
#     for tok in tokens:
#         freqs[tok] += 1

#     vocab = [k for k, v in freqs.items() if (k != '\n')]
#     unk_toks = ["--unk--", "--unk_adj--", "--unk_adv--", "--unk_digit--", "--unk_noun--", "--unk_punct--", "--unk_upper--", "--unk_verb--"]
#     vocab.extend(unk_toks)
#     vocab.append("--n--")
#     vocab.append(" ")
#     vocab = sorted(set(vocab))
#     return vocab

ModuleNotFoundError: No module named 'utils'

In [ ]:
random.seed(1234)
train_set, test_set = train_test_split(wsj,test_size=0.3)

print(len(train_set))
print(len(test_set))

2739
1175


In [ ]:
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

70136

In [ ]:
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['CERTIFICATES',
 'OF',
 'DEPOSIT',
 ':',
 '8.07',
 '%',
 'one',
 'month',
 ';',
 '8.06']

In [ ]:
# vocabulary
V = set(tokens)
print(len(V))

10212


In [ ]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

45

In [ ]:
print(T)

{'IN', 'PRP$', 'JJS', 'WP$', 'RBR', 'JJ', 'CD', 'WDT', '-LRB-', 'VBN', 'VBZ', ':', "''", 'RP', '-NONE-', '#', 'LS', 'NN', 'TO', 'VBD', 'VB', '``', 'EX', 'NNP', 'RB', 'NNPS', 'UH', 'CC', 'PDT', 'POS', ',', '.', 'RBS', 'JJR', 'NNS', '$', 'VBP', 'WP', 'VBG', 'FW', 'PRP', 'DT', 'MD', '-RRB-', 'WRB'}


In [ ]:
train_set[10]

[('The', 'DT'),
 ('government', 'NN'),
 ('includes', 'VBZ'),
 ('money', 'NN'),
 ('spent', 'VBN'),
 ('*', '-NONE-'),
 ('on', 'IN'),
 ('residential', 'JJ'),
 ('renovation', 'NN'),
 (';', ':'),
 ('Dodge', 'NNP'),
 ('does', 'VBZ'),
 ("n't", 'RB'),
 ('*?*', '-NONE-'),
 ('.', '.')]

In [ ]:
def word2features(sent, i):
  word = sent[i][0]
  features = {
    'bias': 1.0,
    'word': word,
    'len(word)': len(word),
    'word[:4]': word[:4],
    'word[:3]': word[:3],
    'word[:2]': word[:2],
    'word[-3:]': word[-3:],
    'word[-2:]': word[-2:],
    'word[-4:]': word[-4:],
    'word.lower()': word.lower(),
    'word.stemmed': re.sub(r'(.{2,}?)([aeiougyn]+$)',r'\1',
    word.lower()),
    'word.ispunctuation': (word in string.punctuation),
    'word.isdigit()': word.isdigit()
  }

  if i > 0:
    word1 = sent[i-1][0]
    features.update({
    '-1:word': word1,
    '-1:len(word)': len(word1),
    '-1:word.lower()': word1.lower(),
    '-1:word.stemmed': re.sub(r'(.{2,}?)
    ([aeiougyn]+$)',r'\1', word1.lower()),
    '-1:word[:3]': word1[:3],
    '-1:word[:2]': word1[:2],
    '-1:word[-3:]': word1[-3:],
    '-1:word[-2:]': word1[-2:],
    '-1:word.isdigit()': word1.isdigit(),
    '-1:word.ispunctuation': (word1 in string.punctuation),
    })
  else:
    features['BOS'] = True
  if i > 1:
    word2 = sent[i-2][0]
    features.update({
    '-2:word': word2,
    '-2:len(word)': len(word2),
    '-2:word.lower()': word2.lower(),
    '-2:word[:3]': word2[:3],
    '-2:word[:2]': word2[:2],
    '-2:word[-3:]': word2[-3:],
    '-2:word[-2:]': word2[-2:],
    '-2:word.isdigit()': word2.isdigit(),
    '-2:word.ispunctuation': (word2 in string.punctuation),
    })
if i < len(sent)-1:
word1 = sent[i+1][0]
features.update({
'+1:word': word1,
'+1:len(word)': len(word1),
'+1:word.lower()': word1.lower(),
'+1:word[:3]': word1[:3],
'+1:word[:2]': word1[:2],
'+1:word[-3:]': word1[-3:],
'+1:word[-2:]': word1[-2:],
'+1:word.isdigit()': word1.isdigit(),
'+1:word.ispunctuation': (word1 in string.punctuation),
})
else:
features['EOS'] = True
if i < len(sent) - 2:
word2 = sent[i+2][0]
features.update({
'+2:word': word2,
'+2:len(word)': len(word2),
'+2:word.lower()': word2.lower(),
'+2:word.stemmed': re.sub(r'(.{2,}?)
([aeiougyn]+$)',r'\1', word2.lower()),
'+2:word[:3]': word2[:3],
'+2:word[:2]': word2[:2],
'+2:word[-3:]': word2[-3:],
'+2:word[-2:]': word2[-2:],
'+2:word.isdigit()': word2.isdigit(),
'+2:word.ispunctuation': (word2 in string.punctuation),
})
return features
def sent2features(sent):
return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
return [word[1] for word in sent]



# New

In [7]:
from nltk.corpus import treebank,brown

corpus = treebank.tagged_sents(tagset = 'universal')
tag_dict={}
word_dict={}

train_set,test_set = train_test_split(corpus,train_size=0.95,test_size=0.05,random_state = 101)

for sent in train_set:
    for elem in sent:
        w = elem[0]
        tag= elem[1]
        if w not in word_dict:
            word_dict[w]=0
        if tag not in tag_dict:
            tag_dict[tag]=0
        word_dict[w]+=1
        tag_dict[tag]+=1

print('Number of words(M): ',len(word_dict))
print('Number of tags(N): ',len(tag_dict))
print(tag_dict)

Number of words(M):  12100
Number of tags(N):  12
{'NOUN': 27352, 'VERB': 12910, 'DET': 8281, 'CONJ': 2151, 'ADV': 3019, '.': 11123, 'X': 6284, 'ADJ': 6069, 'ADP': 9353, 'PRT': 3035, 'PRON': 2612, 'NUM': 3358}


In [8]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics


def word2features(sent,i):
    word = sent[i][0]
    features ={
    'bias': 1.0,
    'word':word,
    'is_first': i == 0,
    'is_last': i == len(sent) - 1,
    'is_capitalized': word[0].upper() == word[0],
    'is_all_caps': word.upper() == word,
    'is_all_lower': word.lower() == word,
    'prefix-1': word[0],
    'prefix-2': word[:2],
    'prefix-3': word[:3],
    'suffix-1': word[-1],
    'suffix-2': word[-2:],
    'suffix-3': word[-3:],
    'prev_word': '' if i == 0 else sent[i - 1][0],
    'next_word': '' if i == len(sent) - 1 else sent[i + 1][0],
    'has_hyphen': '-' in word,
    'is_numeric': word.isdigit(),
    'capitals_inside': word[1:].lower() != word[1:]
    }
                
    return features

def sent2features(sent):
    return [word2features(sent,i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for i,label in sent]

In [9]:
X_train=[sent2features(s) for s in train_set]
y_train=[sent2labels(s) for s in train_set]

X_test=[sent2features(s) for s in test_set]
y_test=[sent2labels(s) for s in test_set]

In [18]:
!pip uninstall scikit-learn
!pip install scikit-learn==0.24

^C
^C
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [19]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

In [ ]:
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)

In [17]:
y_pred = crf.predict(X_test)
labels=list(crf.classes_)

metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.9673129601617199

In [27]:
scorr=[]
for pred_sent_tags,sent_tags in zip(y_pred,y_test):
  for pred,original in zip(pred_sent_tags,sent_tags):
    if(pred==original):
      scorr.append(1)
    else:
      scorr.append(0)

In [28]:
sum(scorr)/len(scorr) *100

96.74400467927471